# PDB 复合物批量分析：蛋白-小分子配体关键结合位点

本 Notebook 用于批量分析 [complex] 目录下的 PDB 复合物结构，自动识别蛋白与小分子配体之间的空间接触残基（候选关键结合位点），并导出为 CSV 结果文件，便于后续统计和可视化。

主要步骤：

1. 环境与依赖检查（Biopython）
2. 设置数据目录和参数（距离阈值、忽略的 HET 名称等）
3. 解析 PDB：区分蛋白原子 (`ATOM`) 与配体原子 (`HETATM`)
4. 计算蛋白残基与配体的最小原子-原子距离，识别接口残基
5. 汇总并导出 `binding_sites.csv`，每行对应一个“蛋白残基–配体拷贝”的接触记录

你可以根据需要调整参数，比如：
- 距离 cutoff（默认 4.0 Å）
- 忽略的 HET 残基列表（溶剂、离子等）
- 是否限制只分析特定的蛋白链 ID

In [ ]:


from pathlib import Path
import math
import csv

from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa

In [ ]:
# 根目录： 
BASE_DIR = Path(r"c:\Users\Administrator\Desktop\IGEM\stage1\notebook-lab")
PDB_DIR = BASE_DIR / "complex-20251129T063258Z-1-001" / "complex"

# 输出结果文件
OUTPUT_CSV = BASE_DIR / "binding_sites.csv"

# 距离阈值（Å），用于定义“接触/关键位点”
DIST_CUTOFF = 4.0

# 忽略的 HET 残基名（溶剂、简单离子等）
IGNORED_HET = {
    "HOH", "WAT",  # 水
    "NA", "K", "CL", "MG", "CA", "ZN", "FE", "MN", "CU",
    "SO4", "PO4", "IOD", "IOD", "GOL", "PEG"
}

# 如果你已有确定的受体链，可以在这里设置，例如 ["A"]
# 如果 None 或空列表，则分析所有蛋白链
RECEPTOR_CHAINS = []  # 例如 ["A"] 或 ["A", "B"]

工具函数：判断蛋白原子与配体原子（Code）

In [5]:
def split_protein_and_ligands(structure):
    """
    将结构中的原子分为：
    - protein_residues: 所有蛋白残基（按链和残基分组）
    - ligand_residues: 所有小分子配体残基（HETATM，排除 IGNORED_HET）
    """
    protein_residues = []
    ligand_residues = []

    for model in structure:
        for chain in model:
            chain_id = chain.id

            # 如果用户限定链，只保留指定的蛋白链
            chain_is_receptor = (
                not RECEPTOR_CHAINS or chain_id in RECEPTOR_CHAINS
            )

            for residue in chain:
                resname = residue.get_resname().strip()
                hetfield = residue.id[0]  # ' ' for standard, 'H_' for HET

                if is_aa(residue, standard=True) and chain_is_receptor:
                    protein_residues.append((chain_id, residue))
                elif hetfield.startswith("H") and resname not in IGNORED_HET:
                    # 认为是小分子配体候选
                    ligand_residues.append((chain_id, residue))

    return protein_residues, ligand_residues

计算最小距离并生成记录（Code）


In [6]:
def compute_contacts_for_structure(pdb_path):
    """
    对单个 PDB 文件：
    - 找出蛋白残基集合和配体残基集合
    - 使用 NeighborSearch 计算残基-配体的最小原子-原子距离
    返回：列表，每个元素是 dict，对应 CSV 中的一行。
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(pdb_path.stem, str(pdb_path))

    protein_residues, ligand_residues = split_protein_and_ligands(structure)

    if not protein_residues or not ligand_residues:
        return []

    # 所有原子用于空间索引
    all_atoms = [atom for atom in structure.get_atoms()]
    ns = NeighborSearch(all_atoms)

    records = []

    for prot_chain_id, prot_res in protein_residues:
        # 预先取出蛋白残基原子列表
        prot_atoms = list(prot_res.get_atoms())

        for lig_chain_id, lig_res in ligand_residues:
            lig_atoms = list(lig_res.get_atoms())

            min_dist = math.inf

            # 粗筛：用邻域搜索限定配体附近的原子，加快速度（可选）
            # 这里只是直接双层循环，数据量允许可以先这样
            for pa in prot_atoms:
                for la in lig_atoms:
                    d = pa - la
                    if d < min_dist:
                        min_dist = d

            if min_dist <= DIST_CUTOFF:
                # 记录一个蛋白残基-配体拷贝的接触
                res_id = prot_res.id[1]
                icode = prot_res.id[2].strip() or ""
                lig_res_id = lig_res.id[1]
                lig_icode = lig_res.id[2].strip() or ""

                record = {
                    "pdb_id": pdb_path.stem,
                    "protein_chain": prot_chain_id,
                    "protein_resnum": res_id,
                    "protein_icode": icode,
                    "protein_resname": prot_res.get_resname().strip(),
                    "ligand_resname": lig_res.get_resname().strip(),
                    "ligand_chain": lig_chain_id,
                    "ligand_resnum": lig_res_id,
                    "ligand_icode": lig_icode,
                    "min_distance": round(float(min_dist), 3),
                }
                records.append(record)

    return records

批量遍历所有 PDB 并导出 CSV（Code）

In [7]:
def analyze_all_pdbs(pdb_dir, output_csv):
    pdb_files = sorted(pdb_dir.glob("*.pdb"))
    print(f"Found {len(pdb_files)} PDB files under {pdb_dir}")

    all_records = []
    for i, pdb_path in enumerate(pdb_files, 1):
        print(f"[{i}/{len(pdb_files)}] Processing {pdb_path.name} ...", end="\r")
        try:
            recs = compute_contacts_for_structure(pdb_path)
            all_records.extend(recs)
        except Exception as e:
            print(f"\nError processing {pdb_path.name}: {e}")

    print(f"\nTotal contact records: {len(all_records)}")

    if not all_records:
        print("No contacts found. Check your parameters (DIST_CUTOFF, IGNORED_HET, RECEPTOR_CHAINS).")
        return

    fieldnames = list(all_records[0].keys())
    with open(output_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(all_records)

    print(f"Saved results to: {output_csv}")


# 运行主函数
analyze_all_pdbs(PDB_DIR, OUTPUT_CSV)

Found 3432 PDB files under c:\Users\Administrator\Desktop\IGEM\stage1\notebook-lab\complex-20251129T063258Z-1-001\complex
[3432/3432] Processing 999.pdb ....
Total contact records: 25626
Saved results to: c:\Users\Administrator\Desktop\IGEM\stage1\notebook-lab\binding_sites.csv
